In [ ]:
import os
import cv2
import numpy as np
from google.colab import drive
from tqdm import tqdm

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Tentukan Path untuk Input dan Output
input_path = "/content/drive/MyDrive/Dataset_Telur_Skripsi"  # Path untuk folder dataset telur
output_path = "/content/drive/MyDrive/Preprocessing_Dataset_Skripsi"

Mounted at /content/drive


In [ ]:
# 3. Folder kategori
classes = ['Berkualitas', 'Kurang Berkualitas', 'Tidak Berkualitas']

# 4. Memastikan folder output ada
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# Fungsi untuk crop citra (fokus pada telur)
def crop_to_egg(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresholded = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    else:
        return image

In [ ]:
# Fungsi untuk menerapkan CLAHE (Contrast Limited Adaptive Histogram Equalization)
def apply_clahe(image):
    # Konversi citra ke grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl_image = clahe.apply(gray)
    # Ubah kembali ke BGR agar tetap dalam format warna
    return cv2.cvtColor(cl_image, cv2.COLOR_GRAY2BGR)

In [ ]:
# Fungsi untuk resize citra
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)

In [ ]:
# Fungsi untuk padding citra (jika diperlukan)
def pad_image(image, target_size=(224, 224)):
    h, w = image.shape[:2]
    top = bottom = (target_size[1] - h) // 2
    left = right = (target_size[0] - w) // 2
    padded_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    return padded_image

In [ ]:
# Fungsi untuk augmentasi citra (flip horizontal dan rotasi 90 derajat)
def augment_image(image):
    # Flip horizontal
    flipped_image = cv2.flip(image, 1)

    # Rotasi 90 derajat
    rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

    return [flipped_image, rotated_image]

In [ ]:
# Fungsi untuk memproses dan menyimpan citra yang telah dipreproses
def preprocess_and_save(input_path, output_path):
    # Memproses semua citra dalam folder
    for kls in classes:
        input_dir = os.path.join(input_path, kls)

        for filename in tqdm(os.listdir(input_dir), desc=f"Proses {kls}"):
            if filename.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(input_dir, filename)
                image = cv2.imread(image_path)

                # Crop citra ke fokus telur
                cropped_image = crop_to_egg(image)

                # Terapkan CLAHE untuk kontras
                clahe_image = apply_clahe(cropped_image)

                # Resize citra
                resized_image = resize_image(clahe_image)

                # Padding citra jika perlu
                padded_image = pad_image(resized_image)

                # Simpan citra yang telah diproses
                preprocessed_class_folder = os.path.join(output_path, kls)
                if not os.path.exists(preprocessed_class_folder):
                    os.makedirs(preprocessed_class_folder)

                preprocessed_image_path = os.path.join(preprocessed_class_folder, filename)
                cv2.imwrite(preprocessed_image_path, padded_image)

                # Lakukan augmentasi dan simpan citra yang sudah diperbesar
                augmented_images = augment_image(padded_image)

                # Mendapatkan informasi usia simpan dan kelas kualitas dari nama file
                usia_simpan = filename.split('_')[0]  # Extract usia simpan berdasarkan nama file (0_1_, 1_2_ etc.)
                kelas_kualitas = determine_quality_class(filename)

                for idx, augmented_image in enumerate(augmented_images):
                    # Penamaan augmentasi yang sesuai
                    augmented_image_path = os.path.join(preprocessed_class_folder, f"{usia_simpan}_aug_{idx}_{filename}")
                    cv2.imwrite(augmented_image_path, augmented_image)

# Fungsi untuk menentukan kelas kualitas berdasarkan usia simpan
def determine_quality_class(filename):
    if filename.startswith(('0_','1_','2_')): return 'Berkualitas'
    elif filename.startswith('3_'):           return 'Kurang Berkualitas'
    else:                                     return 'Tidak Berkualitas'

In [ ]:
# Panggil fungsi untuk preprocessing dan menyimpan citra yang telah diproses
preprocess_and_save(input_path, output_path)

Proses Tidak Berkualitas: 100%|██████████| 270/270 [01:45<00:00,  2.57it/s]
